**W207 Final Project**

Kaggle Competition: San Francisco Crime Classification

Team members:
Chula
Jasen
Shelly
Adam



In [1]:
# Import libraries
import numpy as np
import pandas as pd
import csv as csv 
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_mldata
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report

# Disable new windows for each plot in matplotlib
%matplotlib inline

# Set randomizer seed
np.random.seed(0)

In [19]:
###################
# Data processing #
###################

# Set local directories
local_dir_train = r'C:\Users\aspitzig\Google Drive\Backup -- From 2015\Education\Berkeley\Courses\DATASCI W207 - Applied Machine Learning\Projects\Final Project\train.csv'
local_dir_test = r'C:\Users\aspitzig\Google Drive\Backup -- From 2015\Education\Berkeley\Courses\DATASCI W207 - Applied Machine Learning\Projects\Final Project\test.csv'

# Read data as pandas dataframes
df_train = pd.read_csv(local_dir_train)
df_test = pd.read_csv(local_dir_test)

# Set up train, dev and test datasets, and convert to numpy arrays
train_X = df_train.as_matrix(columns = ["Dates", "DayOfWeek", "PdDistrict", "Address", "X", "Y"])
train_Y = df_train.as_matrix(columns = ["Category"])
test_data = df_test.as_matrix()
num_train = len(train_X)/2                        # NOTE: Train data split in half to create dev data (consider modifying)
train_data, train_labels = train_X[:num_train], train_Y[:num_train]
dev_data, dev_labels = train_X[num_train:], train_Y[num_train:]

# Verify datasets
print "TRAIN"
for data, label in zip(train_data[:10], train_labels[:10]):
    print label, data
print "DEV"
for data, label in zip(dev_data[:10], dev_labels[:10]):
    print label, data
print "TEST"
for data in test_data[:10]:
    print data

print
print "train_data: " + str(train_data.shape)
print "dev_data: " + str(dev_data.shape)
print "test_data: " + str(test_data.shape)

# Notes:
# Original parameters of training dataset: ["Dates", "Category", "Descript", "DayOfWeek", "PdDistrict", "Resolution", "Address", "X", "Y"]
# Original parameters of test dataset: ["Id", "Dates", "DayOfWeek", "PdDistrict", "Address" "X", "Y"]
# To make training and test match, training predictor parameters reduced to: ["Dates", "DayOfWeek", "PdDistrict", "Address", "X", "Y"]
# i.e. "Descript" and "Resolution" predictor paramters were removed from training datset
# Note: test_data still retains the parameter "Id", which appears to be required for Kaggle submission
# ...take this into account when testing model and making final predictions

TRAIN
['WARRANTS'] ['2015-05-13 23:53:00' 'Wednesday' 'NORTHERN' 'OAK ST / LAGUNA ST'
 -122.425891675136 37.7745985956747]
['OTHER OFFENSES'] ['2015-05-13 23:53:00' 'Wednesday' 'NORTHERN' 'OAK ST / LAGUNA ST'
 -122.425891675136 37.7745985956747]
['OTHER OFFENSES'] ['2015-05-13 23:33:00' 'Wednesday' 'NORTHERN' 'VANNESS AV / GREENWICH ST'
 -122.42436302145 37.8004143219856]
['LARCENY/THEFT'] ['2015-05-13 23:30:00' 'Wednesday' 'NORTHERN' '1500 Block of LOMBARD ST'
 -122.42699532676599 37.80087263276921]
['LARCENY/THEFT'] ['2015-05-13 23:30:00' 'Wednesday' 'PARK' '100 Block of BRODERICK ST'
 -122.438737622757 37.771541172057795]
['LARCENY/THEFT'] ['2015-05-13 23:30:00' 'Wednesday' 'INGLESIDE' '0 Block of TEDDY AV'
 -122.40325236121201 37.713430704116]
['VEHICLE THEFT'] ['2015-05-13 23:30:00' 'Wednesday' 'INGLESIDE' 'AVALON AV / PERU AV'
 -122.423326976668 37.7251380403778]
['VEHICLE THEFT'] ['2015-05-13 23:30:00' 'Wednesday' 'BAYVIEW' 'KIRKWOOD AV / DONAHUE ST'
 -122.371274317441 37.727564

In [ ]:
# Feature engineering 
